In [6]:
import jsonpickle
import sqlite3

class KeyValueStore:
    def __init__(self, db_name):
        self.db_name = db_name
        if not self.table_exists():
            self.create_table()

    def table_exists(self):
        conn = sqlite3.connect(self.db_name)
        c = conn.cursor()
        c.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='key_value_store'")
        result = c.fetchone()
        conn.close()
        return result is not None

    def create_table(self):
        conn = sqlite3.connect(self.db_name)
        c = conn.cursor()
        c.execute('''CREATE TABLE key_value_store
                     (key TEXT PRIMARY KEY, serialized_object TEXT)''')
        conn.commit()
        conn.close()

    def __getitem__(self, key):
        conn = sqlite3.connect(self.db_name)
        c = conn.cursor()
        c.execute("SELECT serialized_object FROM key_value_store WHERE key = ?", (key,))
        result = c.fetchone()
        conn.close()
        if result is None:
            raise KeyError(f"Key '{key}' not found")
        serialized_object = result[0]
        deserialized_object = jsonpickle.decode(serialized_object)
        return deserialized_object

    def __setitem__(self, key, value):
        serialized_object = jsonpickle.encode(value)
        conn = sqlite3.connect(self.db_name)
        c = conn.cursor()
        c.execute('''INSERT OR REPLACE INTO key_value_store (key, serialized_object)
                     VALUES (?, ?)''', (key, serialized_object))
        conn.commit()
        conn.close()

    def __delitem__(self, key):
        conn = sqlite3.connect(self.db_name)
        c = conn.cursor()
        c.execute("DELETE FROM key_value_store WHERE key = ?", (key,))
        conn.commit()
        conn.close()


In [7]:
db_name = 'key_value_store.db'
store = KeyValueStore(db_name)

In [2]:
store['example_key'] = 'example_value'  # Set a key-value pair
print(store['example_key'])             # Get value by key
del store['example_key']                # Delete key-value pair

# Verify deletion
try:
    print(store['example_key'])
except KeyError as e:
    print(e)  # Expected: "Key 'example_key' not found"

example_value
"Key 'example_key' not found"


In [12]:
class  Dump:
    attribute=None

In [15]:
a = Dump()
a.attribute = "Testing"
store['class'] = a

In [16]:
store['class'].attribute

'Testing'